In [ ]:
import os

os.environ.update({"TF_CPP_MIN_LOG_LEVEL": "3", "CUDA_VISIBLE_DEVICES": ""})

import numpy as np
import sklearn.metrics
import tensorflow as tf

import chiron


In [ ]:
results_dir = "../results/combined-4-class"
epoch = 10


In [ ]:
model = chiron.utils.load_model(os.path.join(results_dir, "model.json"))
model.load_weights(
    os.path.join(
        results_dir,
        "weights",
        chiron.utils.get_weights_path().format(epoch=epoch),
    )
)
config = chiron.utils.load_yaml(os.path.join(results_dir, "config.yaml"))
image_size = tf.TensorShape(config["image_size"])
model.build([None] + image_size + [3])
model.summary()


In [ ]:
tfrecord_file = "../tfrecord/combined/val.tfrecord"
batch_size = 32

dataset = (
    chiron.tfrecord.load_tfrecord(tfrecord_file)
    .map(chiron.preprocessing.LabelMapper(config["label_map"]))
    .map(chiron.preprocessing.Resizer(image_size))
    .batch(batch_size)
    .map(chiron.preprocessing.PerImageStandardizer())
    .map(chiron.preprocessing.GrayscaleToRgb())
)

y_true = []
y_score = []
for images, labels in dataset:
    y_true.append(tf.one_hot(labels, config["num_classes"]).numpy())
    y_score.append(tf.nn.softmax(model(images, training=False)).numpy())
y_true = np.vstack(y_true)
y_score = np.vstack(y_score)


In [ ]:
sklearn.metrics.average_precision_score(y_true, y_score, average=None)


In [ ]:
sklearn.metrics.roc_auc_score(y_true, y_score, average=None)


In [ ]:
cm = sklearn.metrics.confusion_matrix(
    y_true.argmax(axis=1), y_score.argmax(axis=1)
)
cm


In [ ]:
cm.diagonal() / cm.sum(axis=1)
